# Test Millera-Rabina

#### Autorzy:
1. Mateusz Serek 
2. Weronika Sadoch

### Załadujmy odpowiednie biblioteki

In [ ]:
import random
import ipywidgets as widgets

## Wprowadzenie
Test pierwszości Millera-Rabina służy do sprawdzenia czy zadana liczba jest złożona, czy prawdopodobnie pierwsza.


## Teoria

#### Twierdzenie Eulera
Jeżeli: 
$$ NWD(x,n)=1 $$ 
To: 
$$ x^{\varphi(n)} \equiv 1 \text{ }(mod\text{ }n) $$

## Lista kroków
Dane są $n >= 5$
1. Szukamy takich liczb naturalnych $s, d$: $s \neq d$, gdzie $s$ jest nieparzyste oraz $n - 1  = 2 ^{s}$ * $ d$. Można to wykonać przez 
2. Wybierzmy liczbę losową $a$ taką, że $1 < a < n$
3. Niech $b = a ^{d}$. Jeżeli $b \equiv  1$ (mod $n$), zwróć prawdę.
4. W przeciwnym wypadku jeżeli $b^{2^{r}} \equiv -1$ (mod $n$), gdzie $\forall r$ $1 \le  r \le  k-1$, zwróć prawdę.
5. W przeciwnym wypadku zwróć fałsz

Zwrócona prawda oznacza, że liczba $n$ jest prawdopodobnie pierwsza, w przypadku fałszu liczba $n$ napewno nie jest liczbą pierwszą.

## Usprawnienie testu
Aby Zwiększyć prawdopodobieństwo trzeba zwiększyć liczbę prób wykonanych dla danej liczby.

## Implementacja Testu Millera-Rabina w SageMath

In [ ]:
def miller_rabbin(n: int, k: int) -> bool: 
    #n - testowana liczba
    #k - dokladnosc...
    
    if n < 5:
        raise ValueError(f"n = {n} cannot be below 5")
        
    if k < 0:
        raise ValueError(f"k = {k} cannot be below 0")
        
    if n % 2 == 0:
        return False
    
    d = n - 1
    s = 0
    
    while d % 2 == 0: #Krok 1.
        s += 1
        d /= 2
        
    for _ in range(k):
        a = random.randint(2, n) #Krok 2.
        b = (a ^ d) % n 
        
        for _ in range(s): #Krok 4.
            y = (b ^ 2) % n
            
            if y == 1 and b != 1 and b != n - 1:
                return False
            
            b = y
            
        if y != 1:
            return False
        
    return True

### Sprawdźmy działanie algorytmu.

In [ ]:
print(miller_rabbin(10009729, 1))   
print(miller_rabbin(10009729, 5))   
print(miller_rabbin(10009729, 10))   
print(miller_rabbin(10009729, 20))   

True
True
True
True


## Złożoność obliczeniowa
lorem ipsum 

### Źródła
https://wstein.org/ent/ent.pdf <br>
https://en.wikipedia.org/wiki/Miller–Rabin_primality_test